In [1]:
import numpy as np  
import pandas as pd  
earn_data=pd.read_csv("直播间营收数据.csv",encoding="UTF-8")
fan_data=pd.read_csv("虚拟主播粉丝数据.csv",encoding="UTF-8")
year_liver_data=pd.read_csv("虚拟主播1年数据.csv",encoding="UTF-8")
video_data=pd.read_csv("虚拟主播视频数据.csv",encoding="gbk")
liver_data=pd.read_csv("liver_data.csv",encoding="UTF-8")

In [2]:
popu_data=year_liver_data[year_liver_data['liveCount'] != 0]
liver_data=liver_data[['mid','attracitve']]
live_data=popu_data[['mid','name','fans','guards','archiveView','liveCount','participant','danmaku','income','goldUser']]
live_data=pd.merge(live_data,liver_data,on = ['mid'],how='left')
live_data=live_data[live_data['fans']<=402658 ]
live_data.head()

,mid,name,fans,guards,archiveView,liveCount,participant,danmaku,income,goldUser,attracitve
1,32906095,Alec艾里,84111,33,3103799,7,312,2154,667300,41,16.900324
2,1480838,EricaZehnt_埃莉卡,46510,36,540647,15,2802,23558,20984400,550,11.517861
4,1442771,Kirikosama,86717,91,917083,103,13773,52795,33979100,1029,50.852931
5,668479385,Laine_official,98785,38,5631002,41,13855,62585,30985800,1603,51.779606
6,423902976,Minicatty,97263,106,5661359,117,43812,256594,160746600,3394,68.478261


In [10]:
import time
import re
date2=[]
for index, row in video_data.iterrows():
    time_array  = time.strptime(row['created'], "%Y/%m/%d %H:%M")
    str_date = time.strftime("%Y-%m-%d %H:%M", time_array)
    date1=re.findall(r'(.*) ', str_date)
    date2.append(date1[0])
video_data['DATE']=date2
#video_data.head()

In [4]:
import re
DATE=[]
for index, row in fan_data.iterrows():
    date=re.findall(r'(.*)T', row['created_at'])
    DATE.append(date[0])
fan_data['created_at']=DATE

In [5]:
date1=[]
live_if=[]
for index, row in earn_data.iterrows():
    date=re.findall(r'(.*)T', row['date'])
    date1.append((date[0]))
    if row['currency']>=100: live_if.append(1)
    else: live_if.append(0)
earn_data['DATE']=date1
earn_data['live_if']=live_if
#earn_data.head()

In [6]:
earn_data=earn_data[['mid','count','currency','DATE','live_if']]
video_data=video_data[['mid','length','DATE']]
earn_data1=pd.merge(earn_data,live_data,on='mid',how='right')
earn_data1=pd.merge(earn_data1,video_data,on=['mid','DATE'],how='left')

In [58]:
def looper(limit,cols,df,col):     #剔除p值不显著的
    for i in range(len(cols)):  
        data1 = df[cols]  
        x = sm.add_constant(data1) #生成自变量  
        y = df[col] #生成因变量  
        model = sm.OLS(y, x) #生成模型  
        result = model.fit() #模型拟合  
        pvalues = result.pvalues #得到结果中所有P值  
        pvalues.drop('const',inplace=True) #把const取得  
        pmax = max(pvalues) #选出最大的P值  
        if pmax>limit:  
            ind = pvalues.idxmax() #找出最大P值的index  
            cols.remove(ind) #把这个index从cols中删除  
        else:  
            return result  
#Prob (F-statistic)是我们常用的P值，其接近于零，说明我们的多元线性方程是显著的.
#通过图3中“P>|t|”这一列来判断，这一列中我们可以选定一个阈值，比如统计学常用的就是0.05、0.02或0.01，
#这里我们就用0.05，凡是P>|t|这列中数值大于0.05的自变量，我们都把它剔除掉，这些就是和y线性关系不显著的自变量，

In [67]:
earn_data1.head()

,mid,count,currency,DATE,live_if,name,fans,guards,archiveView,liveCount,participant,danmaku,income,goldUser,attracitve,length
0,32906095,0.0,0.0,2023-01-19,0.0,Alec艾里,84111,33,3103799,7,312,2154,667300,41,16.900324,NaN
1,32906095,22.0,15.6,2023-01-02,0.0,Alec艾里,84111,33,3103799,7,312,2154,667300,41,16.900324,NaN
2,32906095,30.0,151.5,2023-01-01,1.0,Alec艾里,84111,33,3103799,7,312,2154,667300,41,16.900324,NaN
3,32906095,1.0,0.1,2022-12-26,0.0,Alec艾里,84111,33,3103799,7,312,2154,667300,41,16.900324,NaN
4,32906095,99.0,342.4,2022-12-25,1.0,Alec艾里,84111,33,3103799,7,312,2154,667300,41,16.900324,NaN


In [68]:
earn_data1['length']=earn_data1['length'].fillna(0)
earn_data1.head()

,mid,count,currency,DATE,live_if,name,fans,guards,archiveView,liveCount,participant,danmaku,income,goldUser,attracitve,length
0,32906095,0.0,0.0,2023-01-19,0.0,Alec艾里,84111,33,3103799,7,312,2154,667300,41,16.900324,0
1,32906095,22.0,15.6,2023-01-02,0.0,Alec艾里,84111,33,3103799,7,312,2154,667300,41,16.900324,0
2,32906095,30.0,151.5,2023-01-01,1.0,Alec艾里,84111,33,3103799,7,312,2154,667300,41,16.900324,0
3,32906095,1.0,0.1,2022-12-26,0.0,Alec艾里,84111,33,3103799,7,312,2154,667300,41,16.900324,0
4,32906095,99.0,342.4,2022-12-25,1.0,Alec艾里,84111,33,3103799,7,312,2154,667300,41,16.900324,0


In [75]:
video_if=[]
for index, row in earn_data1.iterrows():
    if row['length']==0: video_if.append(0)
    else: video_if.append(1)
earn_data1['video_if']=video_if
earn_data1.head()

,mid,count,currency,DATE,live_if,name,fans,guards,archiveView,liveCount,participant,danmaku,income,goldUser,attracitve,length,video_if
0,32906095,0.0,0.0,2023-01-19,0.0,Alec艾里,84111,33,3103799,7,312,2154,667300,41,16.900324,0,0
1,32906095,22.0,15.6,2023-01-02,0.0,Alec艾里,84111,33,3103799,7,312,2154,667300,41,16.900324,0,0
2,32906095,30.0,151.5,2023-01-01,1.0,Alec艾里,84111,33,3103799,7,312,2154,667300,41,16.900324,0,0
3,32906095,1.0,0.1,2022-12-26,0.0,Alec艾里,84111,33,3103799,7,312,2154,667300,41,16.900324,0,0
4,32906095,99.0,342.4,2022-12-25,1.0,Alec艾里,84111,33,3103799,7,312,2154,667300,41,16.900324,0,0


In [76]:
earn_data1=earn_data1.dropna()

In [77]:
earn_data1.head()

,mid,count,currency,DATE,live_if,name,fans,guards,archiveView,liveCount,participant,danmaku,income,goldUser,attracitve,length,video_if
0,32906095,0.0,0.0,2023-01-19,0.0,Alec艾里,84111,33,3103799,7,312,2154,667300,41,16.900324,0,0
1,32906095,22.0,15.6,2023-01-02,0.0,Alec艾里,84111,33,3103799,7,312,2154,667300,41,16.900324,0,0
2,32906095,30.0,151.5,2023-01-01,1.0,Alec艾里,84111,33,3103799,7,312,2154,667300,41,16.900324,0,0
3,32906095,1.0,0.1,2022-12-26,0.0,Alec艾里,84111,33,3103799,7,312,2154,667300,41,16.900324,0,0
4,32906095,99.0,342.4,2022-12-25,1.0,Alec艾里,84111,33,3103799,7,312,2154,667300,41,16.900324,0,0


In [78]:
import statsmodels.api as sm 
cols=['attracitve', 'video_if','live_if','liveCount','fans','count']
play_result = looper(0.05,cols,earn_data1, 'currency')  
play_result.summary() 

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               currency   R-squared:                       0.407
Model:                            OLS   Adj. R-squared:                  0.407
Method:                 Least Squares   F-statistic:                 1.046e+04
Date:                Fri, 10 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:08:58   Log-Likelihood:            -8.0764e+05
No. Observations:               76199   AIC:                         1.615e+06
Df Residuals:                   76193   BIC:                         1.615e+06
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -195.9767     92.098     -2.128      0.033    -376.488     -15.465
attracitve    -3.5670      0.801     -4.454      0.000      -5.137      -1.997
video_if     425.3742     93.680      4.541      0.000     241.761     608.987
live_if     1426.8079     91.387     15.613      0.000    1247.689    1605.926
fans           0.0032      0.000      7.348      0.000       0.002       0.004
count          1.4359      0.006    227.085      0.000       1.424       1.448
==============================================================================
Omnibus:                   215357.552   Durbin-Watson:                   1.794
Prob(Omnibus):                  0.000   Jarque-Bera (JB):      16977612952.414
Skew:                          37.277   Prob(JB):                         0.00
Kurtosis:                    2314.232   Cond. No.                     4.40e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.4e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [85]:
#import statsmodels.api as sm 
cols=['attracitve', 'video_if','live_if','liveCount','fans']
play_result = looper(0.05,cols,earn_data1,'count')  
play_result.summary() 

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  count   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     40.12
Date:                Fri, 10 Mar 2023   Prob (F-statistic):           6.78e-26
Time:                        15:54:20   Log-Likelihood:            -7.6520e+05
No. Observations:               76199   AIC:                         1.530e+06
Df Residuals:                   76195   BIC:                         1.530e+06
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.8615     56.789     -0.068      0.946    -115.168     107.445
attracitve    -1.0471      0.448     -2.339      0.019      -1.925      -0.170
live_if      530.6046     52.741     10.061      0.000     427.233     633.976
liveCount      1.3211      0.486      2.716      0.007       0.368       2.274
==============================================================================
Omnibus:                   274822.836   Durbin-Watson:                   1.841
Prob(Omnibus):                  0.000   Jarque-Bera (JB):     121453360141.724
Skew:                          74.719   Prob(JB):                         0.00
Kurtosis:                    6186.138   Cond. No.                         358.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [79]:
live_data[live_data['mid']==573930836]

,mid,name,fans,guards,archiveView,liveCount,participant,danmaku,income,goldUser,attracitve
53,573930836,咲间妮娜Official,75365,258,3685637,62,25976,113410,83694700,2966,41.508745


In [11]:
video_data=pd.read_csv("虚拟主播视频数据.csv",encoding="gbk")
monthvideodata=video_data[(video_data['DATE']>='2022-11-01')&(video_data['DATE']<='2022-11-30')][['mid','DATE','is_union_video','video_review','play','comment','length']]
nina_video=monthvideodata[monthvideodata['mid']==573930836]
nina_video

,mid,DATE,is_union_video,video_review,play,comment,length
135338,573930836,2022-11-11,0,3,5124,46,4:35
162855,573930836,2022-11-10,1,6,10155,60,2:39
198070,573930836,2022-11-01,0,13,22728,106,0:18
211001,573930836,2022-11-19,0,16,30381,56,1:31
219050,573930836,2022-11-07,0,23,36957,190,2:00
267946,573930836,2022-11-29,1,309,179974,150,3:10


In [12]:
earn_data[(earn_data['mid']==573930836) & (earn_data['DATE']>='2022-11-01')&(earn_data['DATE']<='2022-11-30')][['mid','count','currency','DATE']]

,mid,count,currency,DATE
9630,573930836,2,2.0,2022-11-30
9631,573930836,1,0.5,2022-11-28
9632,573930836,5,142.0,2022-11-27
9633,573930836,1,0.1,2022-11-26
9634,573930836,486,2015.2,2022-11-25
9635,573930836,382,1112.3,2022-11-24
9636,573930836,47,492.4,2022-11-23
9637,573930836,1,138.0,2022-11-22
9638,573930836,254,924.0,2022-11-21
9639,573930836,114,473.3,2022-11-20
